In [1]:
import os
import cv2
import random
import shutil
import time
import matplotlib
import glob
import operator
import psutil
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from shutil import copyfile
import pandas as pd
import PIL
from mlxtend.plotting import plot_confusion_matrix

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
classify_train_dir = str(input("Classify Train Dir: "))
classify_train = os.path.join(classify_train_dir, 'classify train')
classify_train_lis = []
for file in os.listdir(classify_train):
    print(os.path.join(classify_train, file))
    classify_train_lis.append(os.path.join(classify_train, file))

TRAINING_DIR = classify_train_lis[0]
VALIDATION_DIR = classify_train_lis[1]
TESTING_DIR = classify_train_lis[2]

Classify Train Dir: /home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master (copy)/DATASET
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master (copy)/DATASET/classify train/training
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master (copy)/DATASET/classify train/validation
/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master (copy)/DATASET/classify train/testing


In [5]:
trainX = []

trainY = []

i,j = 0,0
for i in os.listdir(TRAINING_DIR):
    for j in os.listdir(os.path.join(TRAINING_DIR, i)):
        img = os.path.join(os.path.join(TRAINING_DIR, i), j)
        x = cv2.imread(img)
        trainX.append(x)

        
i,j = 0,0
for i in range(1064):
    trainY.append(0)
for j in range(1064):
    trainY.append(1)


In [6]:
trainX = np.array(trainX)
trainY = np.array(trainY)

In [7]:
trainX.shape

(2128, 224, 224, 3)

In [8]:
trainY.shape

(2128,)

In [9]:
trainX_norm = trainX.astype('float32')

trainX_norm = trainX_norm / 255.0

In [10]:
trainX_norm.shape

(2128, 224, 224, 3)

In [ ]:
x = np.concatenate((trainX_norm, testX_norm), axis=0)
y = np.concatenate((trainY, testY), axis=0)

In [ ]:
learning_rate = float(input("Enter the initial learning rate: "))
epoch = int(input("Enter the maximum number of epochs: "))
batch_size = int(input("Enter batch size: "))

In [ ]:
char_name = str(input("Enter name of the characteristics folder: "))

char = os.path.join('/home/rajarshi/Documents/DATASETS/Health/COVID/covid-chestxray-dataset-master/DATASET', char_name)

if not os.path.exists(char):
    os.mkdir(char)
else:
    shutil.rmtree(char)
    os.mkdir(char)

In [ ]:
def lr_schedule(epoch):
    return learning_rate * (0.1 ** int(epoch / 10))
    
best_model_address = os.path.join(char, 'best_model.h5')

callback = [keras.callbacks.LearningRateScheduler(lr_schedule, verbose = 0),
            keras.callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.001, patience = 10, verbose=0, mode = 'min', restore_best_weights = True),
            keras.callbacks.ModelCheckpoint(best_model_address, monitor = 'loss', verbose=0, save_best_only=True, save_weights_only=False, mode = 'min' , period=1)]

In [ ]:
monitor = int(input("Press 1 to monitor Validation Accuracy\nPress 2 to monitor Validation Loss\nPress 3 to monitor Training Accuracy\nPress 4 to monitor Training Loss\n"))
patience = int(input('Enter number of epochs that will produce monitored quantity with no improvement after which training will be stopped: '))


if monitor == 1:
    metric = 'val_accuracy'
    mode = 'max'
    print("\nMONITORING VALIDATION ACCURACY..........\n")

elif monitor == 2:
    metric = 'val_loss'
    mode = 'min'
    print("\nMONITORING VALIDATION LOSS..........\n")

elif monitor == 3:
    metric = 'accuracy'
    mode = 'max'
    print("\nMONITORING TRAINING ACCURACY..........\n")

elif monitor == 4:
    metric = 'loss'
    mode = 'min'
    print("\nMONITORING TRAINING LOSS..........\n")

callback = [keras.callbacks.LearningRateScheduler(lr_schedule, verbose = 0),
            keras.callbacks.EarlyStopping(monitor = metric, min_delta = 0.001, patience = patience, verbose=0, mode = mode, restore_best_weights = True),
            keras.callbacks.ModelCheckpoint(best_model_address, monitor = metric, verbose=0, save_best_only=True, save_weights_only=False, mode = mode , period=1)]

print("\nTraining will stop if Validation Accuracy doesn't show any improvements for " + str(patience) + " epcohs\n")

In [ ]:
def create_model():
    print("\nTRAINING ON A COMPLEX CUSTOM MODEL:-")

    model = keras.models.Sequential()
    for l in range(layer):
        l += 1
        m = (2**l)//2
        for c in range(conv_layer):
            model.add(Conv2D(conv*m, (conv_size, conv_size), padding = 'same', input_shape = dim, activation = activation))
        model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(dense, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(output_layer, activation=output_activation))
    return model

In [ ]:
if len(os.listdir(TRAINING_DIR)) > 2:
    print("This is a " + str(class_no) + "-Class Classification")
    output_activation = 'softmax'
    losses = 'categorical_crossentropy'
    output_layer = class_no
else:
    print("This is a Binary Classification")
    output_activation = 'sigmoid'
    losses = 'binary_crossentropy'
    output_layer = 1

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)

In [ ]:
dense = int(input("Enter the no. of neurons in dense layer: "))
activation = str(input("Enter the activation function: "))
dropout = float(input("Enter the dropout percentage: "))
dropout = dropout/100

layer = int(input("Enter number of layers you want to apply: "))
conv_layer = int(input("Enter number of convolution layers you want to apply: "))
conv = int(input("Enter the no. of filters in the 1st convolution layer: "))
conv_size = int(input("Enter the size of filters: "))
h = int(input("Image Dimension(H or W): "))
w = h
dim = [h,w,3]

In [ ]:
model = create_model()
model.summary()

In [ ]:
from sklearn.model_selection import KFold
train_scores, val_scores, histories = list(), list(), list()

kfold = KFold(10, shuffle=True, random_state=1)

fold_no = 1

for train, test in kfold.split(x, y):
    model = create_model()
    model.compile(optimizer = optimizer, loss = losses, metrics = ['accuracy', 
                                                               tf.keras.metrics.Precision(), 
                                                               tf.keras.metrics.Recall(), 
                                                               tf.keras.metrics.TruePositives(), 
                                                               tf.keras.metrics.TrueNegatives(), 
                                                               tf.keras.metrics.FalsePositives(),
                                                               tf.keras.metrics.FalseNegatives()])
    
    print('------------------------------------------------------------------------')
    print('Training for fold {}...'.format(fold_no))
    
    history = model.fit(x[train], y[train], epochs=epoch, batch_size=batch_size, validation_data=(x[test], y[test]), callbacks = callback, verbose=0)
    
    train_score = model.evaluate(x[train], y[train], verbose=0)
    val_score = model.evaluate(x[test], y[test], verbose=0)
    
    print("\nTraining Acc: {}, Validation Acc: {}".format(train_score[1],val_score[1]))
    
    train_scores.append(train_score)
    val_scores.append(val_score)
    histories.append(history)
    
    fold_no = fold_no + 1

In [ ]:
train_scores

In [ ]:
val_scores

In [ ]:
i = 0
val_acc = []
tr_acc = []

for i in range(len(val_scores)):
    val_acc.append(val_scores[i][1])
    tr_acc.append(train_scores[i][1])

In [ ]:
np.mean(val_acc), np.std(val_acc)

In [ ]:
np.mean(tr_acc), np.std(tr_acc)

In [ ]:
# plot loss
for i in range(len(histories)):
    plt.title('Training Loss')
    plt.plot(histories[i].history['loss'], color='blue')

plt.savefig(os.path.join(char, 'tr_loss.eps'))
plt.show()

In [ ]:
# plot loss
for i in range(len(histories)):
    plt.title('Cross Entropy Loss')
    plt.plot(histories[i].history['val_loss'], color='orange')
    
plt.savefig(os.path.join(char, 'val_loss.eps'))
plt.show()

In [ ]:
for i in range(len(histories)):
    plt.title('Training Accuracy')
    plt.plot(histories[i].history['accuracy'], color='blue')

plt.savefig(os.path.join(char, 'tr_acc.eps'))
plt.show()

In [ ]:
for i in range(len(histories)):
    plt.title('Validation Accuracy')
    plt.plot(histories[i].history['val_accuracy'], color='orange')

plt.savefig(os.path.join(char, 'val_acc.eps'))
plt.show()

In [ ]:
test_class_list = []
for test_name in os.listdir(TESTING_DIR):
    test = os.path.join(TESTING_DIR,test_name)
    test_class_list.append(test)
test_class_list.sort()

In [ ]:
def max_pred(pred):
    pred_categorical = keras.utils.to_categorical(pred)
    if 1 > 2:
        pred_max = np.argmax(pred)
    else:
        pred_max = np.argmax(pred_categorical)
    return pred_max

In [ ]:
y_true = testY
labels = {'covid': 0, 'normal': 1}

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 

model = tf.keras.models.load_model(os.path.join(char, 'best_model.h5'))

y_pred = []
for i in range(len(os.listdir(TRAINING_DIR))):
    for filename in os.listdir(test_class_list[i]):
        file = os.path.join(test_class_list[i], filename)
        img = image.load_img(file, target_size=(224,224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        images = np.vstack([x])

        pred = model.predict(images, batch_size = 10)
        pred_max = max_pred(pred)
        
        y_pred.append(pred_max)
 
print("Calculating CLASSIFICATION REPORT..........:")
classification_report = classification_report(y_true, y_pred, target_names=labels)
print(classification_report)

print("\nCalculating SENSITIVITY & SPECIFICITY..........:")
cm = confusion_matrix(y_true, y_pred)
total = sum(sum(cm))
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

print("sensitivity = {:.4f}".format(sensitivity))
print("specificity = {:.4f}".format(specificity))

In [ ]:
fig, ax = plot_confusion_matrix(conf_mat=cm,
                                colorbar=True,
                                show_absolute=True,
                                show_normed=True)

plt.savefig(os.path.join(char, 'cm.eps'))
plt.show()